<a href="https://colab.research.google.com/github/BMOnoire/low-resource-machine-translation/blob/master/colab_tamil_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
# some check if tf is allright, if GPU: 0 then check Runtime->Change Runtime->GPU
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.4.0
Num GPUs Available:  1


In [1]:
# install stuff and import aachenscript
print("INSTALL fairseq :")
!pip install fairseq
print("\n\n\nINSTALL sacremoses :")
!pip install sacremoses
print("\n\n\nIMPORT aachenscript :")
!git clone https://github.com/GiIbert/aachenscript

INSTALL fairseq :



INSTALL sacremoses :



IMPORT aachenscript :
fatal: destination path 'aachenscript' already exists and is not an empty directory.


In [3]:
!echo $HOME/aachenscript/checkpoints

/root/aachenscript/checkpoints


In [ ]:

!CUDA_VISIBLE_DEVICES=0 fairseq-train aachenscript/data-bin/enta \
    --task translation \
    --batch-size 48 \
    --arch transformer --encoder-layers 5 --decoder-layers 5 --share-decoder-input-output-embed  --encoder-attention-heads 8 --decoder-attention-heads 8 --encoder-embed-dim 512 --decoder-embed-dim 512 \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --lr 1e-4 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --dropout 0.25 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-tokens 4096 \
    --tokenizer moses \
    --eval-bleu \
    --eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' \
    --eval-bleu-detok moses \
    --eval-bleu-remove-bpe \
    --eval-bleu-print-samples \
    --best-checkpoint-metric bleu --maximize-best-checkpoint-metric \
    --save-dir checkpoints \
    --log-format=json --log-interval=10 2>&1 | tee train.log &

2021-01-07 16:55:25 | INFO | fairseq.checkpoint_utils | saved checkpoint checkpoints/checkpoint3.pt (epoch 3 @ 2274 updates, score 0.66) (writing took 58.41841359099999 seconds)
2021-01-07 16:55:25 | INFO | fairseq_cli.train | end of epoch 3 (average epoch stats below)
2021-01-07 16:55:25 | INFO | train | {"epoch": 3, "train_loss": "11.98", "train_nll_loss": "11.318", "train_ppl": "2552.59", "train_wps": "1092.6", "train_ups": "1.42", "train_wpb": "771.4", "train_bsz": "48", "train_num_updates": "2274", "train_lr": "5.685e-05", "train_gnorm": "2.669", "train_train_wall": "250", "train_wall": "1526"}
2021-01-07 16:55:25 | INFO | fairseq.trainer | begin training epoch 4
2021-01-07 16:55:27 | INFO | train_inner | {"epoch": 4, "update": 3.008, "loss": "11.58", "nll_loss": "10.883", "ppl": "1887.98", "wps": "24.8", "ups": "0.03", "wpb": "710.8", "bsz": "48", "num_updates": "2280", "lr": "5.7e-05", "gnorm": "2.507", "train_wall": "3", "wall": "1528"}
2021-01-07 16:55:31 | INFO | train_inner 